# Coauthorship Edgelist ROR to ROR

In [23]:
import pandas as pd
import numpy as np
import sys
from collections import Counter
from os import path
from pyate import combo_basic
from math import log, e
import math

# N.B., need to install dfply ----
from dfply import *


In [2]:
###############################
### Functions
###############################

def normalize_counter(x):
	total = sum(x.values(), 0.0)
	for key in x:
		x[key] /= total
	return x

In [3]:
###############################
### Input Parameters
###############################

# Do we include all authors or just the first/last author -------
input_author_position = "AllAuthors"

if input_author_position == "AuthorFirstLast":
	remove_position_ = "middle"
else:
	remove_position_ = "ANYTHING"

In [17]:
###############################
### Read in File
###############################

being_cited_df = pd.read_csv("INPUT_SQL_Coauthorships_Astronomy_and_Astrophysics.csv")

ror_df = pd.read_csv("INPUT_ROR_Metadata.csv")

In [24]:
ror_df

,id,name,types,status,links,aliases,labels,acronyms,wikipedia_url,established,...,country.country_name,external_ids.GRID.preferred,external_ids.GRID.all,external_ids.ISNI.preferred,external_ids.ISNI.all,external_ids.FundRef.preferred,external_ids.FundRef.all,external_ids.Wikidata.preferred,external_ids.Wikidata.all,relationships
0,https://ror.org/019wvm592,Australian National University,Education,active,http://www.anu.edu.au/,NaN,NaN,ANU,http://en.wikipedia.org/wiki/Australian_Nation...,1946.0,...,Australia,grid.1001.0,grid.1001.0,NaN,0000 0001 2180 7477,5.011000e+11,501100000995;501100001151;100009020,NaN,Q127990,"Related: https://ror.org/041c7s516, https://ro..."
1,https://ror.org/02bfwt286,Monash University,Education,active,http://www.monash.edu/,NaN,NaN,NaN,http://en.wikipedia.org/wiki/Monash_University,1958.0,...,Australia,grid.1002.3,grid.1002.3,NaN,0000 0004 1936 7857,5.011000e+11,501100001779;501100001144;501100007917;5011000...,NaN,Q598841,"Related: https://ror.org/0484pjq71, https://ro..."
2,https://ror.org/00rqy9422,University of Queensland,Education,active,http://www.uq.edu.au/,NaN,NaN,UQ,http://en.wikipedia.org/wiki/University_of_Que...,1909.0,...,Australia,grid.1003.2,grid.1003.2,NaN,0000 0000 9320 7537,5.011000e+11,501100001794;501100008407;501100005268,NaN,Q866012,"Related: https://ror.org/010g47133, https://ro..."
3,https://ror.org/01sf06y89,Macquarie University,Education,active,http://mq.edu.au/,NaN,NaN,NaN,http://en.wikipedia.org/wiki/Macquarie_University,1964.0,...,Australia,grid.1004.5,grid.1004.5,NaN,0000 0001 2158 5405,NaN,501100001230,NaN,Q741082,Related: https://ror.org/0402tt118; Child: htt...
4,https://ror.org/01kj2bm70,Newcastle University,Education,active,http://www.ncl.ac.uk/,University of Newcastle upon Tyne,NaN,NaN,http://en.wikipedia.org/wiki/Newcastle_University,1834.0,...,United Kingdom,grid.1006.7,grid.1006.7,NaN,0000 0001 0462 7212,NaN,501100008406;501100000774,NaN,Q837164,"Related: https://ror.org/00cdwy346, https://ro..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105289,https://ror.org/05s32j989,Sivas Bilim ve Teknoloji Üniversitesi,Education,active,https://www.sivas.edu.tr,Sivas Bilim ve Teknoloji Universitesi,en: Sivas University of Science and Technology,NaN,https://en.wikipedia.org/wiki/Sivas_University...,2018.0,...,Turkey,NaN,NaN,0000 0004 8398 8295,0000 0004 8398 8295,NaN,NaN,Q52162282,Q52162282,NaN
105290,https://ror.org/008hssd09,Regione Lazio,Government,active,https://www.regione.lazio.it,Latium Region,en: Lazio Region,NaN,NaN,NaN,...,Italy,NaN,NaN,0000 0001 1135 4988,0000 0001 1135 4988,5.011000e+11,501100009880,Q1282,Q1282,NaN
105291,https://ror.org/02jqzm779,Atlas Üniversitesi,Education,active,http://www.atlas.edu.tr,Atlas Universitesi; Istanbul Atlas Universitesi,en: Atlas University,NaN,https://en.wikipedia.org/wiki/Atlas_University,2018.0,...,Turkey,NaN,NaN,0000 0004 7863 4273,0000 0004 7863 4273,NaN,NaN,Q52162250,Q52162250,NaN
105292,https://ror.org/013q33h79,Institute of Electronics and Telecommunication...,Facility,active,https://www.ietr.fr,NaN,fr: Institut d'Electronique et de Télécommunic...,IETR,NaN,2002.0,...,France,grid.462104.5,grid.462104.5,NaN,0000 0000 8584 0666,NaN,NaN,Q30261383,Q30261383,"Parent: https://ror.org/04xaa4j22, https://ror..."


In [25]:
being_cited_df

,work_id,ror,country_code
0,https://openalex.org/W100026270,https://ror.org/00cvxb145,US
1,https://openalex.org/W100026270,https://ror.org/00cvxb145,US
2,https://openalex.org/W100026270,https://ror.org/00cvxb145,US
3,https://openalex.org/W100026270,https://ror.org/00cvxb145,US
4,https://openalex.org/W100050316,https://ror.org/01tmp8f25,MX
...,...,...,...
3834872,https://openalex.org/W998121512,https://ror.org/022kthw22,US
3834873,https://openalex.org/W998157275,NaN,BE
3834874,https://openalex.org/W99953471,https://ror.org/05rfqv493,US
3834875,https://openalex.org/W99957615,https://ror.org/01kg8sb98,US


In [7]:
abstracts_df = pd.read_csv("preprocessed_data_Jul14.csv")
abstracts_df

,concept_id,work_id,publication_year,title,paperabstract,country,year_concept,concatenated_title_abstract,US,IN,...,pub_interval_2015_2019,pub_interval_2010_2014,pub_interval_2005_2009,pub_interval_2000_2004,pub_interval_1995_1999,pub_interval_1985_1994,pub_interval_1975_1984,pub_interval_1965_1974,pub_interval_1900_1964,pub_interval_1824_1899
0,https://openalex.org/C44870925,https://openalex.org/W1993867637,2004,KINEMATIC TREATMENT OF CORONAL MASS EJECTION E...,We present a kinematic study of the evolution ...,US US US US,2004+https://openalex.org/C44870925,KINEMATIC TREATMENT OF CORONAL MASS EJECTION E...,100.000000,0.0,...,0,0,0,1,0,0,0,0,0,0
1,https://openalex.org/C44870925,https://openalex.org/W2022503540,1991,The imaging performance of the Hubble Space Te...,Problems with the HST instantaneous imaging pe...,US,1991+https://openalex.org/C44870925,The imaging performance of the Hubble Space Te...,100.000000,0.0,...,0,0,0,0,0,1,0,0,0,0
2,https://openalex.org/C44870925,https://openalex.org/W2077626606,2003,Numerical Simulations of Mass Outflows Driven ...,"We study the two-dimensional, time-dependent m...",US,2003+https://openalex.org/C44870925,Numerical Simulations of Mass Outflows Driven ...,100.000000,0.0,...,0,0,0,1,0,0,0,0,0,0
3,https://openalex.org/C1276947,https://openalex.org/W3104346433,2003,The Origin of X-shaped Radio Galaxies: Clues f...,Existing radio images of a few X-shaped radio ...,IN US,2003+https://openalex.org/C1276947,The Origin of X-shaped Radio Galaxies: Clues f...,50.000000,50.0,...,0,0,0,1,0,0,0,0,0,0
4,https://openalex.org/C44870925,https://openalex.org/W2081151530,2002,The Ghost of Sagittarius and Lumps in the Halo...,We identify new structures in the halo of the ...,US US US US,2002+https://openalex.org/C44870925,The Ghost of Sagittarius and Lumps in the Halo...,100.000000,0.0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63994,https://openalex.org/C44870925,https://openalex.org/W3102168087,2003,Accretion Disk Spectra of the Ultra-luminous X...,Ultra-luminous Compact X-ray Sources (ULXs) in...,US CH JP,2003+https://openalex.org/C44870925,Accretion Disk Spectra of the Ultra-luminous X...,33.333333,0.0,...,0,0,0,1,0,0,0,0,0,0
63995,https://openalex.org/C1276947,https://openalex.org/W3098886327,2008,The ghost of a dwarf galaxy: fossils of the hi...,We present an extragalactic perspective of an ...,GB US,2008+https://openalex.org/C1276947,The ghost of a dwarf galaxy: fossils of the hi...,50.000000,0.0,...,0,0,1,0,0,0,0,0,0,0
63996,https://openalex.org/C44870925,https://openalex.org/W3093754290,2020,Observations of the Solar Corona from Space,Space observations of the atmosphere of the Su...,IT IT,2020+https://openalex.org/C44870925,Observations of the Solar Corona from Space Sp...,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0
63997,https://openalex.org/C44870925,https://openalex.org/W3121322575,2011,Constraints on the topology of the Universe de...,We impose constraints on the topology of the U...,CH CH,2011+https://openalex.org/C44870925,Constraints on the topology of the Universe de...,0.000000,0.0,...,0,1,0,0,0,0,0,0,0,0


In [8]:
# Merge the DataFrames on 'work_id
merged_df = pd.merge(being_cited_df, abstracts_df[['work_id', 'publication_year']], on='work_id', how='left')
being_cited_df['publication_year'] = merged_df['publication_year']
being_cited_df

,work_id,ror,country_code,publication_year
0,https://openalex.org/W100026270,https://ror.org/00cvxb145,US,NaN
1,https://openalex.org/W100026270,https://ror.org/00cvxb145,US,NaN
2,https://openalex.org/W100026270,https://ror.org/00cvxb145,US,NaN
3,https://openalex.org/W100026270,https://ror.org/00cvxb145,US,NaN
4,https://openalex.org/W100050316,https://ror.org/01tmp8f25,MX,NaN
...,...,...,...,...
3834872,https://openalex.org/W998121512,https://ror.org/022kthw22,US,NaN
3834873,https://openalex.org/W998157275,NaN,BE,NaN
3834874,https://openalex.org/W99953471,https://ror.org/05rfqv493,US,NaN
3834875,https://openalex.org/W99957615,https://ror.org/01kg8sb98,US,NaN


In [9]:
print(being_cited_df['publication_year'].isna().sum())


2579402


In [10]:
non_nan_publication_years = being_cited_df[being_cited_df['publication_year'].notna()]
non_nan_publication_years

,work_id,ror,country_code,publication_year
326,https://openalex.org/W1028656435,https://ror.org/04t5xt781,US,2000.0
327,https://openalex.org/W1028656435,https://ror.org/041nk4h53,US,2000.0
328,https://openalex.org/W1028656435,https://ror.org/042nb2s44,US,2000.0
329,https://openalex.org/W1028656435,https://ror.org/01nrxwf90,GB,2000.0
330,https://openalex.org/W1028656435,https://ror.org/047426m28,US,2000.0
...,...,...,...,...
3834786,https://openalex.org/W992191593,https://ror.org/046rm7j60,US,1993.0
3834787,https://openalex.org/W992191593,https://ror.org/005ta0471,IT,1993.0
3834788,https://openalex.org/W992191593,https://ror.org/046rm7j60,US,1993.0
3834789,https://openalex.org/W992191593,https://ror.org/05hy3tk52,FR,1993.0


In [16]:
mean_publication_year = being_cited_df['publication_year'].mean()
print(mean_publication_year)

median_publication_year = being_cited_df['publication_year'].median()
print(median_publication_year)

mode_publication_year = being_cited_df['publication_year'].mode()[0]
print(mode_publication_year)

count_before_2000 = being_cited_df[being_cited_df['publication_year'] < 2000].shape[0]
print(count_before_2000)

count_2000_and_after = being_cited_df[being_cited_df['publication_year'] >= 2000].shape[0]
print(count_2000_and_after)

count_before_1950 = being_cited_df[being_cited_df['publication_year'] < 1950].shape[0]
print(count_before_1950)

count_1950_and_after = being_cited_df[being_cited_df['publication_year'] >= 1950].shape[0]
print(count_1950_and_after)

2007.0832194985962
2007.0
2007.0
140162
1115313
161
1255314


In [27]:
###############################
### Prepare File
###############################
# to count number of co-authorship per country

being_cited_df["country_code"] = being_cited_df["country_code"].astype(str)

country_list = being_cited_df.country_code.drop_duplicates().tolist()
country_list.sort()

being_cited_dict = {country: being_cited_df.query("country_code == @country") for country in country_list}

In [28]:
###############################
### Loop through each year in discipline
###############################

citation_list = []
full_citation_df = pd.DataFrame()

# Loop through each country
for country, data in being_cited_dict.items():
    try:
        coauthorship_count = len(data)
        citation_list.append([country, coauthorship_count])
    except:
        continue

full_citation_df = pd.DataFrame(citation_list, columns=["Country", "Coauthorship_Count"])

In [29]:
###############################
### Combine with ROR DataFrame
###############################

full_citation_with_ROR_df = pd.merge(full_citation_df, ror_df[['country.country_name', 'id', 'name']].rename(columns={'country.country_name': 'Country_Name', 'id': 'ROR_ID', 'name': 'Institution_Name'}), left_on='Country', right_on='Country_Name', how='left')
full_citation_with_ROR_df = full_citation_with_ROR_df.drop(columns=['ROR_ID', 'Country_Name', 'Institution_Name'])


full_citation_with_ROR_df

,Country,Coauthorship_Count
0,AE,1745
1,AF,1
2,AL,4
3,AM,2086
4,AO,22
...,...,...
134,YE,6
135,ZA,15682
136,ZM,3
137,ZW,17


In [30]:
network_data = []
grouped = being_cited_df.groupby('work_id')

for name, group in grouped:
    rors = group['ror'].unique()

    # Counting occurrences for normalization
    pair_counts = Counter()
    for i in range(len(rors)):
        for j in range(i+1, len(rors)):
            sender = rors[i]
            receiver = rors[j]
            pair_counts[(sender, receiver)] += 1

    # Assign weights
    for (sender, receiver), count in pair_counts.items():
        weight = 1 / count
        existing = next((item for item in network_data if item['Sender'] == sender and item['Receiver'] == receiver), None)
        if existing:
            existing['Weight'] += weight
        else:
            network_data.append({'Sender': sender, 'Receiver': receiver, 'Weight': weight})

network_df = pd.DataFrame(network_data)

network_df = pd.merge(network_df, ror_df[['id', 'name']], left_on='Sender', right_on='id', how='left')
network_df.rename(columns={'name': 'Sender_Name'}, inplace=True)
network_df.drop('id', axis=1, inplace=True)

network_df = pd.merge(network_df, ror_df[['id', 'name']], left_on='Receiver', right_on='id', how='left')
network_df.rename(columns={'name': 'Receiver_Name'}, inplace=True)
network_df.drop('id', axis=1, inplace=True)

In [22]:
network_df

,Sender,Receiver,Weight,id,Sender_Name
0,https://ror.org/03ht1xw27,https://ror.org/00za53h95,15.0,https://ror.org/03ht1xw27,Tata Institute of Fundamental Research
1,NaN,https://ror.org/02gfc7t72,1.0,NaN,NaN
2,NaN,https://ror.org/0384j8v12,1.0,NaN,NaN
3,https://ror.org/02gfc7t72,https://ror.org/0384j8v12,133.0,https://ror.org/02gfc7t72,Spanish National Research Council
4,https://ror.org/01qtasp15,https://ror.org/01c27hj86,65.0,https://ror.org/01qtasp15,European Southern Observatory
...,...,...,...,...,...
441997,NaN,https://ror.org/02p77k626,1.0,NaN,NaN
441998,NaN,https://ror.org/010pmpe69,1.0,NaN,NaN
441999,NaN,https://ror.org/05kt9ap64,1.0,NaN,NaN
442000,NaN,https://ror.org/03a64bh57,1.0,NaN,NaN


In [ ]:
###############################
### Output final file to CSV
###############################

# N.B., need to update file name --------------
full_citation_with_ROR_df.to_csv("edge_list_Coauthorship.csv")
